In [ ]:
import numpy as np
import os 
import pickle 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import matplotlib.pyplot as plt

from utils import impute_head_tail_zeros

# 1 Read and preprocess data

In [ ]:
processed_dir = '###YOUR_DIRECTORY###/ArgoverseDatasets/train/processed' # data processed by QCNet 
processed_file_names = [name for name in os.listdir(processed_dir) if
                        os.path.isfile(os.path.join(processed_dir, name)) and
                        name.endswith(('pkl', 'pickle'))]
print('Numbers of file: {}'.format(len(processed_file_names)))

num_historical_steps = 50

all_trajectory = []
         
for i in tqdm(range(len(processed_file_names))):
    process_path = os.path.join(processed_dir, processed_file_names[i])
    with open(process_path, 'rb') as handle:
        data = pickle.load(handle)

    # imputation
    trajectory = impute_head_tail_zeros(data['agent']['position'][:, :, :2])
    theta = impute_head_tail_zeros(data['agent']['heading'][:, :, None])[:, :, 0]
    
    # normalization
    origin = trajectory[:, num_historical_steps - 1, 0:2]
    _theta = theta[:, num_historical_steps - 1]
    cos, sin = _theta.cos(), _theta.sin()
    rot_mat = _theta.new_zeros(data['agent']['num_nodes'], 2, 2)
    rot_mat[:, 0, 0] = cos
    rot_mat[:, 0, 1] = -sin
    rot_mat[:, 1, 0] = sin
    rot_mat[:, 1, 1] = cos
    norm_trajectory = torch.bmm(trajectory - origin[:, :2].unsqueeze(1), rot_mat)

    all_trajectory.append(norm_trajectory)


In [ ]:
# for i in range(len(norm_trajectory)):
#     plt.plot(norm_trajectory[i, :, 0], norm_trajectory[i, :, 1])
# plt.axis('equal')

In [ ]:
all_trajectory_arr = torch.concat(all_trajectory)
print(all_trajectory_arr.shape)

# 2 Network

In [ ]:
class TrajectoryEncoder(nn.Module):
    def __init__(self, C1, C2, H):
        """
        Encoder with 1D CNN and LSTM.

        Args:
            C1 (int): Number of output channels for the first CNN layer
            C2 (int): Number of output channels for the second CNN layer
            H (int): Hidden size of the LSTM (bottleneck feature size)
        """
        super().__init__()
        self.cnn1 = nn.Conv1d(2, C1, kernel_size=3, padding=1)  # Input channels = 2 (x, y)
        self.cnn2 = nn.Conv1d(C1, C2, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(C2, H, num_layers=1, batch_first=True)

    def forward(self, x):
        # x: (batch_size, T, 2)
        x = x.permute(0, 2, 1)  # (batch_size, 2, T) for Conv1d
        x = F.relu(self.cnn1(x))  # (batch_size, C1, T)
        x = F.relu(self.cnn2(x))  # (batch_size, C2, T)
        x = x.permute(0, 2, 1)  # (batch_size, T, C2) for LSTM
        _, (h_n, _) = self.lstm(x)  # h_n: (1, batch_size, H)
        encoded_feature = h_n[0]  # (batch_size, H)
        return encoded_feature

class TrajectoryDecoder(nn.Module):
    def __init__(self, H):
        """
        Decoder with LSTM to reconstruct the trajectory.

        Args:
            H (int): Hidden size of the LSTM (matches encoder bottleneck)
        """
        super().__init__()
        self.lstm = nn.LSTM(2, H, num_layers=1, batch_first=True)  # Input size = 2 (x, y)
        self.linear = nn.Linear(H, 2)  # Map LSTM output to (x, y)

    def forward(self, encoded_feature, decoder_input):
        # encoded_feature: (batch_size, H)
        # decoder_input: (batch_size, T, 2)
        h_0 = encoded_feature.unsqueeze(0)  # (1, batch_size, H)
        c_0 = torch.zeros_like(h_0)  # (1, batch_size, H)
        output, _ = self.lstm(decoder_input, (h_0, c_0))  # (batch_size, T, H)
        pred_traj = self.linear(output)  # (batch_size, T, 2)
        return pred_traj

class TrajectoryAutoencoder(nn.Module):
    def __init__(self, C1, C2, H):
        """
        Autoencoder combining encoder and decoder.

        Args:
            C1 (int): Channels for first CNN layer
            C2 (int): Channels for second CNN layer
            H (int): Hidden size (bottleneck feature dimension)
        """
        super().__init__()
        self.encoder = TrajectoryEncoder(C1, C2, H)
        self.decoder = TrajectoryDecoder(H)

    def forward(self, x):
        # x: (batch_size, T, 2)
        encoded_feature = self.encoder(x)
        # Prepare decoder input with teacher forcing: [0, x[:-1]]
        batch_size = x.size(0)
        start_token = torch.zeros(batch_size, 1, 2, device=x.device)
        decoder_input = torch.cat([start_token, x[:, :-1, :]], dim=1)  # (batch_size, T, 2)
        pred_traj = self.decoder(encoded_feature, decoder_input)
        return pred_traj

# 3 Dataloader

In [ ]:
# Custom Dataset for trajectories
class TrajectoryDataset(Dataset):
    def __init__(self, trajectories):
        """
        Args:
            trajectories (torch.Tensor): Tensor of shape [n, T, 2] containing all trajectories.
        """
        self.trajectories = trajectories

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        return self.trajectories[idx]

# 4 Train AutoEncoder

In [ ]:
C1 = 16  # CNN channels
C2 = 32  # CNN channels
H = 64   # Bottleneck feature size
batch_size = 1024
epochs = 5
learning_rate = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training with DataLoader
# Validation is not performed for simplicity
def train_autoencoder(trajectories, batch_size=32, epochs=50, C1=16, C2=32, H=64, lr=0.001):
    
    # Create dataset and dataloader
    dataset = TrajectoryDataset(trajectories)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    
    # Initialize model, loss, and optimizer
    model = TrajectoryAutoencoder(C1, C2, H).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for batch_traj in dataloader:
            batch_traj = batch_traj.to(device)  # Shape: [batch_size, T, 2]
            
            # Forward pass
            optimizer.zero_grad()
            pred_traj = model(batch_traj)
            loss = criterion(pred_traj, batch_traj)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * batch_traj.size(0)
        
        epoch_loss = running_loss / len(dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    
    return model


trained_model = train_autoencoder(
    all_trajectory_arr, 
    batch_size=batch_size, 
    epochs=epochs)

# 5 Cluster

In [ ]:
from sklearn.cluster import KMeans

# Extract features
trained_model.eval()
encoded_features_list = []

# Process data in batches
with torch.no_grad(): 
    num_trajectories = all_trajectory_arr.shape[0]
    for start_idx in tqdm(range(0, num_trajectories, batch_size)):
        end_idx = min(start_idx + batch_size, num_trajectories)
        batch_traj = all_trajectory_arr[start_idx:end_idx]  # Shape: (batch_size, sequence_length, feature_dim)
        batch_traj = batch_traj.to(device)
        batch_features = trained_model.encoder(batch_traj)  # Shape: (batch_size, H)
        batch_features = batch_features.cpu().numpy()
        encoded_features_list.append(batch_features)

# Concatenate all features
encoded_features = np.concatenate(encoded_features_list, axis=0)  # Shape: (num_trajectories, H)

print(f"Extracted features shape: {encoded_features.shape}")

# Hierarchical K-means (example with 2 levels)
K1 = 5  # Number of clusters at level 1
K2 = 5  # Number of clusters at level 2

# Level 1 clustering
kmeans_l1 = KMeans(n_clusters=K1, n_init='auto', random_state=42)
labels_l1 = kmeans_l1.fit_predict(encoded_features)  # Shape: (num_samples,)

# Level 2 clustering within each level 1 cluster
# Initialize the global labels for the second hierarchy
num_samples = len(encoded_features)
global_labels_l2 = np.zeros(num_samples, dtype=int)  # Unified label array for level 2

# Keep track of the starting label index for each level 1 cluster's subclusters
label_offset = 0

for i in range(K1):
    # Extract features for the current level 1 cluster
    cluster_mask = (labels_l1 == i)
    cluster_features = encoded_features[cluster_mask]
    cluster_size = len(cluster_features)
    
    if cluster_size > K2:
        # Perform level 2 clustering
        kmeans_l2 = KMeans(n_clusters=K2, n_init='auto', random_state=42)
        sub_labels = kmeans_l2.fit_predict(cluster_features)  # Local labels: [0, K2-1]
        
        # Map local subcluster labels to global labels
        # For cluster i, labels should be in range [label_offset, label_offset + K2 - 1]
        global_sub_labels = sub_labels + label_offset
        global_labels_l2[cluster_mask] = global_sub_labels
    else:
        # If too few points, assign a default label (use the first label in the range for this cluster)
        global_labels_l2[cluster_mask] = label_offset
    
    # Update the label offset for the next level 1 cluster
    label_offset += K2

# Verify the label range
print(f"Level 1 labels range: {labels_l1.min()} to {labels_l1.max()}")
print(f"Level 2 labels range: {global_labels_l2.min()} to {global_labels_l2.max()}")


In [ ]:
print(len(labels_l1), len(global_labels_l2))

In [ ]:
hierarchical_labels = torch.stack([
    torch.tensor(labels_l1),
    torch.tensor(global_labels_l2)
], dim=-1)
hierarchical_labels.shape

# 6 Save files

In [ ]:
processed_dir = '###YOUR_DIRECTORY###/ArgoverseDatasets/train/processed' # data processed by QCNet 
processed_file_names = [name for name in os.listdir(processed_dir) if
                        os.path.isfile(os.path.join(processed_dir, name)) and
                        name.endswith(('pkl', 'pickle'))]
print('Numbers of file: {}'.format(len(processed_file_names)))

scene_cluster_dict = {}

start_idx = 0
for i in tqdm(range(len(processed_file_names))):
    process_path = os.path.join(processed_dir, processed_file_names[i])
    with open(process_path, 'rb') as handle:
        data = pickle.load(handle)

    end_idx = start_idx + data['agent']['num_nodes']
    scenario_id = data['scenario_id']
    scene_cluster_dict[scenario_id] = hierarchical_labels[start_idx:end_idx]
    start_idx = end_idx

assert start_idx == len(hierarchical_labels)

In [ ]:
pt_path = 'results/scene_cluster_dict.pt'
torch.save(scene_cluster_dict, pt_path)

print(f"Saved scene_cluster_dict to {pt_path}")

In [ ]:
# Load the cluster dictionary
# pt_path = 'results/scene_cluster_dict.pt'
# loaded_dict = torch.load(pt_path, weights_only=True)
# print(f"Loaded scene_cluster_dict with {len(loaded_dict)} entries")